In [9]:
import pandas as pd
import numpy as np

# 그래프 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

# 통계 관련 라이브러리
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [11]:
df = pd.read_csv("data/member_transaction.csv")

print(df.shape)
df.head(3)

(64000, 10)


,id,recency,zip_code,is_referral,channel,conversion,num_item,total_amount,avg_price,count
0,906145,10,Surburban,0,Phone,0,3.333333,31333.33333,14016.66667,3
1,184478,6,Rural,1,Web,0,4.000000,29000.00000,7250.00000,1
2,394235,7,Surburban,1,Web,0,4.000000,20500.00000,5125.00000,2


# 1. 데이터 전처리

In [12]:
# 컬럼별 결측치 탐지
df.isna().sum() 

id              0
recency         0
zip_code        0
is_referral     0
channel         0
conversion      0
num_item        0
total_amount    0
avg_price       0
count           0
dtype: int64

In [13]:
#df.info() - object  float64 int64 확인가능 컬럼/ 데이터탐색
df_ad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily Time Spent on Site  1000 non-null   float64
 1   Age                       1000 non-null   float64
 2   Area Income               1000 non-null   float64
 3   Daily Internet Usage      1000 non-null   float64
 4   Ad Topic Line             1000 non-null   object 
 5   City                      1000 non-null   object 
 6   Male                      1000 non-null   int64  
 7   Country                   1000 non-null   object 
 8   Timestamp                 1000 non-null   object 
 9   Clicked on Ad             1000 non-null   int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 78.3+ KB


In [14]:
# 평균으로 age 결측치 채워넣기
#age_avg = round(df_ad['Age'].mean(), 0)
#age_avg

np.float64(36.0)

In [15]:
# age 결측치 평균 나이로 채우기
#df_ad['Age'] = df_ad['Age'].fillna(age_avg)
#df_ad

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,36.0,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,3/27/2016 0:53,0
1,80.23,31.0,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,4/4/2016 1:39,0
2,69.47,26.0,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,3/13/2016 20:35,0
3,74.15,29.0,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,1/10/2016 2:31,0
4,68.37,35.0,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,6/3/2016 3:36,0
...,...,...,...,...,...,...,...,...,...,...
995,72.97,30.0,71384.57,208.58,Fundamental modular algorithm,Duffystad,1,Lebanon,2/11/2016 21:49,1
996,51.30,45.0,67782.17,134.42,Grass-roots cohesive monitoring,New Darlene,1,Bosnia and Herzegovina,4/22/2016 2:07,1
997,51.63,51.0,42415.72,120.37,Expanded intangible solution,South Jessica,1,Mongolia,2/1/2016 17:24,1
998,55.55,19.0,41920.79,187.95,Proactive bandwidth-monitored policy,West Steven,0,Guatemala,3/24/2016 2:35,0


In [ ]:
Daily Time Spent on Site → 사이트 일일 체류 시간
Age → 나이
Area Income → 지역 소득
Daily Internet Usage → 일일 인터넷 사용 시간
Ad Topic Line → 광고 제목(문구)
City → 도시
Male → 남성 여부
Country → 국가
Timestamp → 기록 시점(시간)
Clicked on Ad → 광고 클릭 여부

# STEP 1. 통계 검정 광고 채널별 클릭률 차이 → ANOVA

In [ ]:
# id → 고객 ID
# recency → 최근 구매 후 경과 기간
# zip_code → 우편번호(지역 코드)
# is_referral → 추천/지인 소개 여부
# channel → 유입 채널
# conversion → 전환 여부(구매 성공 여부)
# num_item → 구매 상품 개수
# total_amount → 총 구매 금액
# avg_price → 평균 상품 가격
# count → 구매 횟수

In [16]:
df.head(3)

,id,recency,zip_code,is_referral,channel,conversion,num_item,total_amount,avg_price,count
0,906145,10,Surburban,0,Phone,0,3.333333,31333.33333,14016.66667,3
1,184478,6,Rural,1,Web,0,4.000000,29000.00000,7250.00000,1
2,394235,7,Surburban,1,Web,0,4.000000,20500.00000,5125.00000,2


In [19]:
df['channel'].value_counts()

channel
Web             28217
Phone           28021
Multichannel     7762
Name: count, dtype: int64

In [20]:
df.groupby('channel')['conversion'].mean()

channel
Multichannel    0.171734
Phone           0.127155
Web             0.159407
Name: conversion, dtype: float64

In [21]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.ols('conversion ~ C(channel)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

                 sum_sq       df          F        PR(>F)
C(channel)    20.125147      2.0  80.547037  1.155633e-35
Residual    7995.011791  63997.0        NaN           NaN
